# Ćwiczenie 3

Celem ćwiczenia jest imlementacja metody [Minimax z obcinaniem alpha-beta](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) do gry Connect Four (czwórki).

W trakcie ćwiczenia można skorzystać z reposytorium z implementacją gry [Connect Four udostępnionym przez Jakuba Łyskawę](https://github.com/lychanl/two-player-games). Ewentualnie, można zaimplementować samemu grę Connect Four (ale, tak aby rozwiązanie miało ten sam interfejs co podany poniżej).

Implementację Minimax należy przetestować używając różną głębokość przeszukiwania. Implementacja Solvera musi zapewniać interfejs jak poniżej, ale można dodać dowolne metody prywatne oraz klasy wspomagające (jeżeli będą potrzebne).

Punktacja:
- Działająca metoda Minimax - **2 pkt**
- Działająca metoda Minimax z obcinaniem alpha-beta - **1.5 pkt**
- Analiza jakości solvera w zależności od głębokości przeszukiwania **1.5pkt**
    - należy zaimplementować w tym celu prostą wizualizację rozgrywki dwóch agentów, bądź kilka przykładów 'z ręki'
- Jakość kodu **2pkt**

Aby importowanie elementów z poniższej komórki działało należy umieścić tego notebooka w tym samym folderze co paczkę `two_player_games`:
```
├── LICENSE
├── README.md
├── minimax.ipynb # <<< HERE
├── test
│   ├── __init__.py
│   ├── test_connect_four.py
│   ├── test_dots_and_boxes.py
│   └── test_pick.py
└── two_player_games
    ├── __init__.py
    ├── games
    │   ├── connect_four.py
    │   └── dots_and_boxes.py
    ├── move.py
    ├── player.py
    └── state.py
```

In [74]:
from typing import Tuple, List
from copy import copy

from two_player_games.player import Player
from two_player_games.games.connect_four import ConnectFour, ConnectFourMove

Wielkość planszy

In [75]:
ROW_COUNT = 5
COLUMN_COUNT = 6

In [76]:
from copy import deepcopy


class MinMaxSolver:
    def __init__(self, game: ConnectFour):
        self.game = game

    def evaluate_position(self, player: Player)->float:
        return self._get_payoff(player) if self._is_terminal_position() else self.evaluate_heuristic(player)
    
    def evaluate_heuristic(self, player: Player)->float:
        oponent = self.game.first_player if self.game.first_player != player else self.game.second_player

        player_score = \
            self._evaluate_middle_position_heuristic(player) + \
            self._evaluate_check_heuristic(player)
        
        oponent_score = \
            self._evaluate_middle_position_heuristic(oponent) + \
            self._evaluate_check_heuristic(oponent)

        return player_score - oponent_score


    def get_best_move(self)->int:
        pass

    def is_valid_move(self, col_index:int)->bool:
        return col_index in [move.column for move in self.game.get_moves()]

    def minimax(self, depth, alpha:float, beta:float, is_maximizing_player:bool)-> Tuple[int, float]:
        """Returns column index and score"""
        if depth == 0 or self.game.is_finished():
            return None, self.evaluate_position(self.game.first_player)
        
        if is_maximizing_player:
            max_score = float('-inf')
            max_best_move = None
            for move in self.game.get_moves():
                current_state = deepcopy(self.game.state)
                self.game.make_move(move)
                _, new_max_score = self.minimax(depth - 1, alpha, beta, False)
                self.game.state = current_state
                if new_max_score > max_score:
                    max_score = new_max_score
                    max_best_move = move.column
                
                alpha = max(alpha,  max_score)
                if  alpha >= beta:
                    break

            return max_best_move, max_score

        else:
            min_score = float('inf')
            min_best_move = None
            for move in self.game.get_moves():
                current_state = deepcopy(self.game.state)
                self.game.make_move(move)
                _, new_min_score = self.minimax(depth - 1, alpha, beta, True)
                self.game.state = current_state
                if new_min_score < min_score:
                    min_score = new_min_score
                    min_best_move = move.column
                
                beta = min(beta, min_score)
                if  alpha >= beta:
                    break
        
            return min_best_move, min_score
        
    # Helper functions

    def _is_terminal_position(self)->bool:
        return self.game.is_finished()

    def _get_payoff(self, player: Player)->int:
        winner = self.game.get_winner()
        if winner is None:
            return 0    # draw
    
        if self.game.first_player == winner:
            return float('inf')    # player won
        
        return float('-inf')   # player lost
    
    def _evaluate_middle_position_heuristic(self, player: Player)->float:
        middle_columns = [len(self.game.state.fields) // 2 - 1, len(self.game.state.fields) // 2]
        middle_pieces_counts = [self.game.state.count_pieces(col, player) for col in middle_columns]
        return sum(self._score_middle_pieces(count) for count in middle_pieces_counts)
    
    def _evaluate_check_heuristic(self, player: Player) -> float:
        checks = self.game.state.count_checks(player)
        return sum(self._score_check(check) for check in checks)
    
    def _score_middle_pieces(self, count: int) -> int:
        return 2 ** (count - 1) if count > 0 else 0
    
    def _score_check(self, check:  int) -> int:
        return 4**(check - 1) if check > 0 else 0
    


In [77]:
import random
from IPython.display import clear_output

## Rozgrywka

### Rozpoczęcie od środka

In [78]:
# Static test

p1 = Player("a")
p2 = Player("b")

game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
solver = MinMaxSolver(game)

oponent_move, score = solver.minimax(5, float('-inf'), float('inf'), True)
game.make_move(ConnectFourMove(oponent_move))

print(game)
print(f'A should move to {oponent_move} column with score: {score}')

Current player: b
[  ][  ][  ][  ][  ][  ]
[  ][  ][  ][  ][  ][  ]
[  ][  ][  ][  ][  ][  ]
[  ][  ][  ][  ][  ][  ]
[a][  ][  ][  ][  ][  ]
A should move to 0 column with score: 0


### Znajdowanie wygranej

In [79]:
# Static test

p1 = Player("a")
p2 = Player("b")

game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
solver = MinMaxSolver(game)

game.make_move(ConnectFourMove(2))
game.make_move(ConnectFourMove(3))

game.make_move(ConnectFourMove(5))
game.make_move(ConnectFourMove(3))

game.make_move(ConnectFourMove(4))
game.make_move(ConnectFourMove(3))

game.make_move(ConnectFourMove(0))

oponent_move, score = solver.minimax(5, float('-inf'), float('inf'), False)
print(game)
print(f'B should move to {oponent_move} column with score: {score}')

Current player: b
[  ][  ][  ][  ][  ][  ]
[  ][  ][  ][  ][  ][  ]
[  ][  ][  ][b][  ][  ]
[  ][  ][  ][b][  ][  ]
[a][  ][a][b][a][a]
B should move to 3 column with score: -inf


### Blokowanie przeciwnika

In [80]:
# Static test

p1 = Player("a")
p2 = Player("b")

game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
solver = MinMaxSolver(game)

game.make_move(ConnectFourMove(2))
game.make_move(ConnectFourMove(3))

game.make_move(ConnectFourMove(2))
game.make_move(ConnectFourMove(4))

game.make_move(ConnectFourMove(2))
game.make_move(ConnectFourMove(5))

oponent_move, score = solver.minimax(7,  float('-inf'), float('inf'), False)

print(game)
print(f'B should move to {oponent_move} column with score: {score}')

Current player: a
[  ][  ][  ][  ][  ][  ]
[  ][  ][  ][  ][  ][  ]
[  ][  ][a][  ][  ][  ]
[  ][  ][a][  ][  ][  ]
[  ][  ][a][b][b][b]
B should move to 2 column with score: -inf


In [81]:
# Realtime test

p1 = Player("a")
p2 = Player("b")

game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
solver = MinMaxSolver(game)

def print_game():
    clear_output()
    print(game)


column = random.randint(0, COLUMN_COUNT - 1)
print_game()
for i in range(4):
    # player move
    game.make_move(ConnectFourMove(column))

    # oponent move
    oponent_move_column, oponent_score = solver.minimax(5, float('-inf'),  float('inf'),  True)
    if oponent_move_column is None:
        print("No move")
        break

    game.make_move(ConnectFourMove(oponent_move_column))
    print_game()


Current player: a
[  ][a][  ][  ][  ][  ]
[  ][b][  ][  ][  ][  ]
[b][a][  ][  ][  ][  ]
[b][a][  ][  ][  ][  ]
[b][a][  ][  ][  ][  ]


### Symulacja dwóch agentów

In [82]:
def simulate(depth: int, first_move: int = None, verbose: bool = False):
    p1 = Player("a")
    p2 = Player("b")

    game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
    solver = MinMaxSolver(game)

    def print_game():
        clear_output()
        print(game)

    # first player move
    game.make_move(ConnectFourMove(first_move if first_move else random.randint(0, COLUMN_COUNT - 1)))

    if verbose:
        print_game()

    while not game.is_finished():
        # oponent move
        oponent_move_column, oponent_score = solver.minimax(depth, float('-inf'),  float('inf'),  False)
        if oponent_move_column is None:
            oponent_move_column = random.choice(game.get_moves()).column  # lost game

        game.make_move(ConnectFourMove(oponent_move_column))

        if verbose:
            print_game()

        if game.is_finished():
            break

        # player move
        player_move_column, player_score = solver.minimax(depth, float('-inf'),  float('inf'),  True)
        if player_move_column is None:
            player_move_column = random.choice(game.get_moves()).column # lost game
        
        game.make_move(ConnectFourMove(player_move_column))

    return game.get_winner()

In [93]:
# Interactive simulation
depth = 5
winner = simulate(depth, verbose=True)

if winner is None:
    print("Draw")
else:
    print(f"Winner is {winner.char}")

Current player: a
[b][a][a][b][b][b]
[a][b][b][a][a][a]
[b][a][a][b][b][b]
[a][b][b][a][b][a]
[b][a][a][b][a][a]
Draw


In [102]:
# Results test
for depth in range(1, 8):
    winner = simulate(depth)
    print(f'With depth {depth} result was: {winner.char if winner else "draw"}')

With depth 1 result was: draw
With depth 2 result was: draw
With depth 3 result was: a
With depth 4 result was: draw
With depth 5 result was: draw
With depth 6 result was: a
With depth 7 result was: draw


### Gameplay

In [ ]:
p1 = Player("a")
p2 = Player("b")
game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
solver = MinMaxSolver(game)

def print_game():
    clear_output()
    print(game)


while not game.is_finished():
    if game.get_current_player() == p1:
        move = None
        while True:
            try:
                move = int(input("Your turn: "))
            except:
                print("Invalid move")
                continue
            
            if solver.is_valid_move(move):
                break
            else:
                print("Invalid move")
        game.make_move(ConnectFourMove(move))
    else:
        move, score = solver.minimax(5, float('-inf'),  float('inf'),  False)
        if move is None:
            print("No move")
            break
        game.make_move(ConnectFourMove(move))
    print_game()

    
winner = game.get_winner()
if winner is None:
    print("Draw")
else:
    print("Winner:", winner.char)